Clean up the database tables, create new variables as needed, rename variables as needed, text process user location, etc. 

In [19]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
%load_ext sql
%sql mysql://sywang@localhost/?charset=utf8mb4

In [2]:
database = 'sywang_project'
msgs_table = 'msgs'

In [3]:
%%sql 
use {database}; 
show tables; 

 * mysql://sywang@localhost/?charset=utf8mb4
0 rows affected.
76 rows affected.


Tables_in_sywang_project
ags_cp
ags_freq
feat$1gram$msgs$message_id$16to16
feat$1gram$msgs$thread_id$16to16
feat$1gram$msgs$user_id$16to16
feat$1gram$msgs$user_id$16to16$0_01
feat$1gram$msgs$user_id$16to16$0_02
feat$1gram$msgs$user_id$16to16$0_03
feat$1gram$msgs$user_id$16to16$0_05
feat$1gram$msgs$user_id$16to16$0_1


Most of the sql is run directly in HeidiSql but I save the commands here for documentation purposes. For some reason sql that runs on HeidiSql doesn't always run in Jupyter for some reason

In [14]:
#let's make a number of replies column 
%%sql 
drop table threads if exists; 
create table threads as SELECT sywang.ags_threadhtml.thread_id, title, reply_num FROM sywang.ags_threadhtml, (select thread_id, MAX(NUMBER)-1 AS reply_num from msgs 
GROUP BY thread_id) replynumtable
WHERE sywang.ags_threadhtml.thread_id = replynumtable.thread_id; 

SyntaxError: invalid syntax (<ipython-input-14-95279b999f28>, line 3)

Investigate users table. 
Turns out there are only 14 bios. 
But everyone has a location. 

In [21]:
%%sql 
select count(*) from users; 

 * mysql://sywang@localhost/?charset=utf8mb4
1 rows affected.


count(*)
398


In [20]:
%%sql 
select bio from users where bio not in ("None") limit 5; 
select count(bio) from users where bio not in ("None"); 

 * mysql://sywang@localhost/?charset=utf8mb4
5 rows affected.
1 rows affected.


count(bio)
14


In [22]:
%%sql
select count(location) from users where location not in ("None"); 

 * mysql://sywang@localhost/?charset=utf8mb4
1 rows affected.


count(location)
398


In [27]:
%%sql 
update users 
set state = substr(location, -2);  

 * mysql://sywang@localhost/?charset=utf8mb4
10 rows affected.


"substr(location, -2)"
IN
WI
NY
CT
MA
AZ
NY
NY
BC
MD


In [28]:
%sql select * from users limit 10;

 * mysql://sywang@localhost/?charset=utf8mb4
10 rows affected.


user_id,user,profileurl,location,bio,marker,crawled,state
1,Yara Catoira-Boyle,https://www.americanglaucomasociety.net/network/members/profile?UserKey=4ca20dc8-8b67-4ed1-a160-8d0efb58c82a,"Indianapolis, IN",None,0,1,IN
2,Jeffrey Kalenak,https://www.americanglaucomasociety.net/network/members/profile?UserKey=26410406-88f4-440c-bf48-6f2fdf486696,"Milwaukee, WI",None,0,1,WI
3,Nalini Madiwale,https://www.americanglaucomasociety.net/network/members/profile?UserKey=0e40a9d9-4697-45a0-a5a0-8306f5719a5a,"Schenectady, NY",None,0,1,NY
4,James Murphy,https://www.americanglaucomasociety.net/network/members/profile?UserKey=d1da4d47-faba-4114-85d5-43f604aa32c2,"Stamford, CT","Glaucoma-specialist in Connecticut, USA, performing all traditional and MIGS glaucoma surgeries, femto- and Premium IOL/refractive cataracts.Â Big outdoorsman on the weekends!",0,1,CT
5,Peter Zacharia,https://www.americanglaucomasociety.net/network/members/profile?UserKey=fc2cadce-bf75-4edc-810f-c2fea633dcb6,"Worcester, MA",None,0,1,MA
6,George Reiss,https://www.americanglaucomasociety.net/network/members/profile?UserKey=fe774e89-998b-421b-992f-34be2a719397,"Glendale, AZ",None,0,1,AZ
7,Asher Weiner,https://www.americanglaucomasociety.net/network/members/profile?UserKey=dadb71fe-1c33-46cf-a824-5dac9c095f52,"Williamsville, NY",None,0,1,NY
8,Daniel Laroche,https://www.americanglaucomasociety.net/network/members/profile?UserKey=ebe86aa5-e20a-4692-b524-589e023ba0e7,"New York, NY",None,0,1,NY
9,Robert Schertzer,https://www.americanglaucomasociety.net/network/members/profile?UserKey=26866cd9-3bd0-416a-9007-27fed9c6da49,"Vancouver, BC",None,0,1,BC
10,Mona Kaleem,https://www.americanglaucomasociety.net/network/members/profile?UserKey=ab1606c3-0c6c-42f4-8969-ccf7541dfe8b,"Bethesda, MD",None,0,1,MD


Take care of Canadian provinces - set them to "INTL" 

other countries manually reviewed and set to "INTL". Of note we will also count VI (virgin islands) and AE (overseas military) as international

gender also manually reviewed and set to 1 if female, 0 otherwise 

In [ ]:
%%sql 
UPDATE users 
SET state = "INTL" 
WHERE state IN ("ON", "BC", "AB", "MB", "NB", "NL", "NT", "NS", "NU", "PE", "QC", "SK", "YT", "VI", "AE"); 

In [ ]:
%%sql 
UPDATE users 
SET division = "NewEngland"
WHERE state IN ("CT", "MA", "ME", "NH", "RI", "VT"); 
UPDATE users 
SET division = "MidAtlantic"
WHERE state IN ("NJ", "NY", "PA"); 
UPDATE users 
SET division = "MidwestNorth"
WHERE state IN ("IL", "IN", "MI", "OH", "WI"); 
UPDATE users 
SET division = "MidwestWest"
WHERE state IN ("IA", "KS", "MN", "MO", "NE", "ND", "SD"); 
UPDATE users 
SET division = "SouthAtlantic"
WHERE state IN ("DE", "DC", "FL", "GA", "MD", "NC", "SC", "VA", "WV"); 
UPDATE users 
SET division = "SouthEast"
WHERE state IN ("AL", "KY", "MS", "TN"); 
UPDATE users 
SET division = "SouthWest"
WHERE state IN ("AR", "LA", "OK", 'TX'); 
UPDATE users 
SET division = "Mountain"
WHERE state IN ("AZ", "CO", "ID", "MT", "NM", "NV", "UT", "WY"); 
UPDATE users
SET division = "Pacific"
WHERE state IN ("CA", "AL", "HI", "OR", "WA"); 
UPDATE division
SET region = "International"
WHERE state IN ('INTL'); 

In [ ]:
%%sql 
UPDATE users 
SET region = "Northeast"
WHERE division IN ("NewEngland", "MidAtlantic"); 
UPDATE users 
SET region = "Midwest"
WHERE division IN ("MidwestNorth", "MidwestWest"); 
UPDATE users 
SET region = "South"
WHERE division IN ("SouthAtlantic", "SouthEast", "SouthWest"); 
UPDATE users 
SET region = "West"
WHERE division IN ("Mountain", "Pacific"); 
UPDATE users 
SET division = "INTL"
WHERE division IN ("INTL"); 

Let's move the user_id into the messages table so we don't have to join across username, we can join and index across a numeric user_id variable instead

In [ ]:
UPDATE msgs 
SET user_id = (SELECT users.user_id FROM users WHERE msgs.user = users.user); 

Create a new table that is only the titles and first message of the thread, called threadheads. 

Note that this creates a thread_id variable which matches threads.thread_id, and also creates a copy of this variable called message_id, because DLATK likes there to be a message_id key in this sort of table 

In [ ]:
%%sql 
CREATE TABLE threadheads AS 
(SELECT threads.thread_id, msgs.thread_id as message_id, CONCAT(threads.title, " ", msgs.message) AS message 
 FROM threads, msgs WHERE threads.thread_id = msgs.thread_id AND msgs.number=1); 